In [1]:
import numpy as np
from tensorflow.keras import layers
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Lambda, Input, Dense, ReLU, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

In [4]:
import tensorflow as tf
#定义了一个名为VAE_BN的类，继承自object类
class VAE_BN:

    #这是类的构造函数，接受三个参数：nSpecFeatures（光谱特征的数量），intermediate_dim（中间层的维度），和latent_dim（潜在空间的维度）。
    def __init__(self, nSpecFeatures, intermediate_dim, latent_dim):
        self.nSpecFeatures = nSpecFeatures
        self.intermediate_dim = intermediate_dim
        self.latent_dim = latent_dim
    #定义了一个名为sampling的方法，该方法实现了重参数化技巧，这是VAE中的一个关键步骤，用于从编码器的输出中采样。
    #在sampling方法中，args是一个包含两个元素的元组，分别是z_mean（潜在空间的均值）和z_log_var（潜在空间的对数方差）。
    def sampling(self, args):
        z_mean, z_log_var = args
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        #这里使用Keras后端函数tf.random_normal生成一个形状为(batch_size, latent_dim)的正态分布随机矩阵epsilon。
        epsilon = tf.random.normal(shape=(batch, dim))  # random_normal (mean=0 and std=1)
        #通过重参数化技巧，从正态分布中采样，并返回潜在空间的样本。
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


In [8]:
import tensorflow as tf

class VAE_BN:
    # ... 其他类定义 ...

    def get_architecture(self):
        # =========== 1. Encoder Model================
        #定义了输入数据的形状，其中self.nSpecFeatures是类的一个属性，表示输入特征的数量。
        input_shape = (self.nSpecFeatures, )
        #创建了一个Keras的Input层，它是编码器的输入层，具有指定的形状和名称。
        inputs = tf.keras.Input(shape=input_shape, name='encoder_input')
        #添加了一个全连接层（Dense），其神经元数量由self.intermediate_dim指定，并将输入数据传递给它。
        h = tf.keras.layers.Dense(self.intermediate_dim)(inputs)
        #在上一层的输出上应用批量归一化（BatchNormalization），这有助于提高训练的稳定性和速度。
        h = tf.keras.layers.BatchNormalization()(h)
        #应用ReLU激活函数，它对输入数据的每个元素应用非线性操作，保留正数元素，并将负数元素置为零。
        h = tf.keras.layers.ReLU()(h)
        #添加了另一个全连接层，其神经元数量由self.latent_dim指定，用于输出潜在空间的均值，并命名为z_mean。
        z_mean = tf.keras.layers.Dense(self.latent_dim, name='z_mean')(h)
        #对z_mean层的输出应用批量归一化。
        z_mean = tf.keras.layers.BatchNormalization()(z_mean)
        #添加了另一个全连接层，同样输出潜在空间的维度，但这次用于输出潜在空间的对数方差，并命名为z_log_var。
        z_log_var = tf.keras.layers.Dense(self.latent_dim, name='z_log_var')(h)
        #对z_log_var层的输出应用批量归一化。
        z_log_var = tf.keras.layers.BatchNormalization()(z_log_var)
        # ... 可能需要返回或进一步处理这些层 ...


In [9]:
import tensorflow as tf

class VAE_BN:
    # ... 其他类定义 ...

    def get_architecture(self):
        # ... 之前的编码器定义 ...

        # 解码器定义
        latent_inputs = tf.keras.Input(shape=(self.latent_dim,), name='Latent_Space')
        hdec = tf.keras.layers.Dense(self.intermediate_dim)(latent_inputs)
        hdec = tf.keras.layers.BatchNormalization()(hdec)
        hdec = tf.keras.layers.ReLU()(hdec)
        outputs = tf.keras.layers.Dense(self.nSpecFeatures, activation='sigmoid')(hdec)
        decoder = tf.keras.Model(latent_inputs, outputs, name='decoder')
        print("==== Decoder Architecture...")
        decoder.summary()

        # ... 可能需要返回或进一步处理这些层 ...



In [ ]:
import tensorflow as tf

class VAE_BN:
    # ... 其他类定义 ...

    def get_architecture(self):
        # ... 之前的编码器和解码器定义 ...

        # 构建VAE模型
        outputs = self.decoder(self.encoder(inputs)[2])
        self.VAE_BN_model = tf.keras.Model(inputs, outputs, name='VAE_BN')

        # ====== Cost Function (Variational Lower Bound)  ==============
        "KL-div (regularizes encoder) and reconstruction loss (of the decoder): see equation(3) in our paper"
        # 1. KL-Divergence:
        kl_Loss = 1 + self.z_log_var - tf.square(self.z_mean) - tf.exp(self.z_log_var)
        kl_Loss = tf.reduce_sum(kl_Loss, axis=-1)
        kl_Loss *= -0.5
        # 2. Reconstruction Loss
        reconstruction_loss = tf.keras.losses.binary_crossentropy(inputs, outputs)  # Use sigmoid at output layer
        reconstruction_loss *= self.nSpecFeatures

        # ========== Compile VAE_BN model ===========
        model_Loss = tf.reduce_mean(reconstruction_loss + kl_Loss)
        self.VAE_BN_model.add_loss(model_Loss)
        self.VAE_BN_model.compile(optimizer='adam')
        return self.VAE_BN_model, self.encoder

# 示例用法
vae_bn = VAE_BN(nSpecFeatures=..., intermediate_dim=..., latent_dim=...)
vae_bn_model, encoder = vae_bn.get_architecture()
